# 中古音對應閩南語文音查詢系統
### 資料提取

  
  　　這份文件是當初用來把資料以爬蟲抓下來的，所需的資料共有兩份，一個是整理好的<<廣韻>>表格、一個是閩南語每個字文讀音的列表。這兩份資料雖說平常，但網路上卻找不到可下載的檔案；在教育部線上閩南語常用辭典當中有附各個字的讀音表，但沒有將聲、韻、調分置，使用起來實在不便。所以最後決定用爬蟲從 **[韻典網](https://ytenx.org/kyonh/ "廣韻" )** 和 **[教育部臺灣閩南語常用詞辭典](https://sutian.moe.edu.tw/zh-hant/)** 將網頁表格抓下來。

> #### **警告**
> 這裡的每一個爬蟲都必須執行非常久，非必要其實可以直接跳過，抓下的檔案已附在資料夾中。

In [69]:
import requests
import re
import numpy as np
import time
import pandas as pd
from io import StringIO
from bs4 import BeautifulSoup
import matplotlib
import matplotlib.font_manager
import matplotlib.pyplot as plt
import plotly.graph_objects as go

# font setting
#!gdown 1lv0-HBUhnM0rg7rOWDYWPUp93WB5xwsM -O taipei_sans_tc_beta.ttf
matplotlib.font_manager.fontManager.addfont('taipei_sans_tc_beta.ttf')
matplotlib.rc('font', family = 'Taipei Sans TC Beta')

In [659]:
# 韻典網/廣韻/小韻表
blk = 'https://ytenx.org'
tulk = 'https://ytenx.org/kyonh/sieux'
tulk2 = ''

Gtdf0 = pd.DataFrame()
Gtndf0 = pd.DataFrame(columns=['字', '小韻'])

Gtu = requests.get(tulk)


while Gtu.status_code == 200:
    
    # 提取小韻表
    Gtsp = BeautifulSoup(Gtu.text, 'html.parser')
    Gttb = Gtsp.find_all('table')[0]
    Gtdf= pd.read_html(StringIO(str(Gttb)))[0]

    # 尋找每個小韻中的字
    for i in Gttb.find_all('tr'):
        if i.find_all('td') != []:
            href = i.find_all('td')[1].find('a').get('href')
        else:
            continue
        Gtun = requests.get(blk+href)
        if Gtun.status_code == 200:
            Gtnsp = BeautifulSoup(Gtun.text, 'html.parser')
            Gtntb = Gtnsp.find_all('table')[-1]
            Gtndf= pd.DataFrame(pd.read_html(StringIO(str(Gtntb)))[0][0])
            Gtndf.columns = ['字']
            Gtndf['小韻'] = i.find_all('td')[1].find('a').text
        Gtndf0 = pd.concat([Gtndf0,Gtndf],ignore_index=True) 

    # 整理小韻表
    Gtdf0 = pd.concat([Gtdf0,Gtdf],ignore_index=True)

    # 尋找下一頁
    tulk2 = BeautifulSoup(Gtu.text, 'html.parser').find_all('ul')[-1].find_all('a')[-1].get('href')  
    if tulk2 == '':
        break
    time.sleep(1)
    Gtu = requests.get(tulk+tulk2)
else:
    print("請求失敗")

# join
G = pd.merge(Gtndf0, Gtdf0, on='小韻', how='inner')
G.head(20)

# 移除除了聲、韻、調以外的欄位
del G['次序']
del G['反切']
    
# 當小韻是破音字而重複使用時，會有重複的字
G = G.drop_duplicates() 

# 將DataFrame保存為CSV檔
G.to_csv('sieux.csv', index=False)
G.tail(20)

# 59m 41.5s


,字,小韻,聲母,韻目,調,等,呼,韻母
26544,䁆,腌,影,業,入,三,開,業
26545,殜,殜,以,業,入,三,開,業
26546,𩐱,殜,以,業,入,三,開,業
26547,跲,跲,羣,業,入,三,開,業
26548,㭘,跲,羣,業,入,三,開,業
26549,昅,跲,羣,業,入,三,開,業
26550,极,跲,羣,業,入,三,開,業
26551,笈,跲,羣,業,入,三,開,業
26552,乏,乏,並,乏,入,三,合,乏
26553,泛,乏,並,乏,入,三,合,乏


In [167]:
# 教育部閩南語常用字典
Blk = 'https://sutian.moe.edu.tw/'
mnlk = 'https://sutian.moe.edu.tw/zh-hant/siannuntiau/'
href = re.sub(rf'{Blk}', '', mnlk)

kaotian = pd.DataFrame()

Mn = requests.get(mnlk)
while requests.get(Blk+href).status_code== 200:
    Mnsp = BeautifulSoup(Mn.text, 'html.parser')
    Mnul = Mnsp.find_all('ul')[-2]

    # 依調>韻母>聲母順序搜尋
    for i in Mnul.find_all('a')[:-1]:
        # 抓出調
        tiao = re.findall(r'\d',i.text)[0]

        # 搜尋同調之韻母
        href = i.get('href')
        Mnul2 = requests.get(Blk+href)
        Mnsp2 = BeautifulSoup(Mnul2.text, 'html.parser')
        Mnul3 = Mnsp2.find_all('ul')[4]
            
        for j in Mnul3.find_all('a'):
            # 抓出韻母
            un = re.sub(r'\W','', j.text)

            # 搜尋同韻母之聲母
            href = j.get('href')
            Mntr = requests.get(Blk+href)
            Mntr = BeautifulSoup(Mntr.text, 'html.parser')
            Mntr = Mntr.find_all('tr')

            for k in Mntr:
                # 抓出聲母
                if k.find_all('td') == []:
                    continue
                siann = re.sub(rf'{un}', '', k.find_all('td')[0].text)
                # 抓出字
                Mndf = pd.DataFrame([i.text for i in k.find_all('a')], columns = ['字'])

                # 找出文白屬性
                wb = []
                for l in k.find_all('a'):
                    href = l.get('href')
                    Mna = requests.get(Blk+href)
                    Mna = BeautifulSoup(Mna.text, 'html.parser')
                    Mna = Mna.find('table').find('span', class_="tiongkuahho-giapa")
                    if Mna == None:
                        wb.append('')
                        continue
                    wb.append(Mna.text)
                    
                Mndf['閩南語聲母'] = siann
                Mndf['閩南語韻母'] = un
                Mndf['閩南語調'] = tiao
                Mndf['文白音'] = wb
                kaotian = pd.concat([kaotian,Mndf],ignore_index=True)

                time.sleep(1)

        print(f'第{tiao}調完成')

    break
else:
    print("請求失敗")

# 消除聲調記號
kaotian['閩南語韻母'] = [re.sub(r'á|à|â|ā|a̍', 'a', i) for i in kaotian['閩南語韻母']]
kaotian['閩南語韻母'] = [re.sub(r'é|è|ê|ē|e̍', 'e', i) for i in kaotian['閩南語韻母']]
kaotian['閩南語韻母'] = [re.sub(r'í|ì|î|ī|i̍', 'i', i) for i in kaotian['閩南語韻母']]
kaotian['閩南語韻母'] = [re.sub(r'ó|ò|ô|ō|o̍', 'o', i) for i in kaotian['閩南語韻母']]
kaotian['閩南語韻母'] = [re.sub(r'ú|ù|û|ū|u̍', 'u', i) for i in kaotian['閩南語韻母']]
kaotian['閩南語韻母'] = [re.sub(r'ḿ|m̀|m̂|m̄|m̍', 'm', i) for i in kaotian['閩南語韻母']]
kaotian['閩南語韻母'] = [re.sub(r'ń|ǹ|n̂|n̄|n̍', 'n', i) for i in kaotian['閩南語韻母']]

# 將DataFrame保存為CSV檔
kaotian.to_csv('kaotian.csv', index=False)
kaotian.tail(20)

# 約三小時


第1調完成
第2調完成
第3調完成
第4調完成
第5調完成
第7調完成
第8調完成


,字,閩南語聲母,閩南語韻母,閩南語調,文白音
6300,罰,h,uat,8,
6301,劣,l,uat,8,
6302,拔,p,uat,8,文
6303,菝,p,uat,8,替
6304,跋,p,uat,8,
6305,鈸,p,uat,8,文
6306,絕,ts,uat,8,文
6307,奪,t,uat,8,
6308,悅,,uat,8,
6309,曰,,uat,8,
